<font color="green">*To start working on this notebook, or any other notebook that we will use in the Moringa Data Science Course, we will need to save our own copy of it. We can do this by clicking File > Save a Copy in Drive. We will then be able to make edits to our own copy of this notebook.*</font>

# SQL Programming - Subqueries using Select Statements

## 1.0 Connecting to our Database

In [3]:
# We will first load the sql extension into our environment
%load_ext sql

# Then connect to our in memory sqlite database
# NB: This database will cease to exist as soon as the database connection is closed. 
%sql sqlite://

'Connected: @None'

## 1.1 Loading Data from CSV Files

Datasets used in this notebook:


1.   **World Countries**

> ([Dataset Download Link](https://drive.google.com/a/moringaschool.com/file/d/1XlabneVWhvbmpRUi_Wix2FR5uYAAp-ZY/view?usp=sharing))

2.   **World Cities** 

> ([Dataset Download Link](https://drive.google.com/a/moringaschool.com/file/d/1R5952m-8hH91uiwDK8dcL9sT2D_PEcuw/view?usp=sharing))






In [4]:
# Importing Pandas
import pandas as pd
import csv

In [5]:
# Loading our Cities dataset
# 
with open('world_cities.csv','r') as f:
    cities = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql DROP TABLE if EXISTS cities;
%sql PERSIST cities;
%sql SELECT * FROM cities LIMIT 5;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


city,country,latitude,longitude,temperature
Aalborg,Denmark,57.03,9.92,7.52
Aberdeen,United Kingdom,57.17,-2.08,8.1
Abisko,Sweden,63.35,18.83,0.2
Adana,Turkey,36.99,35.32,18.67
Albacete,Spain,39.0,-1.87,12.62


In [6]:
# Loading our cities dataset 
# 
with open('world_countries.csv','r') as f:
    countries = pd.read_csv(f, index_col=0, encoding='utf-8')
%sql DROP TABLE if EXISTS countries;
%sql PERSIST countries;
%sql SELECT * FROM countries LIMIT 5;

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


country,population,EU,coastline
Albania,2.9,no,yes
Andorra,0.07,no,no
Austria,8.57,yes,no
Belarus,9.48,no,no
Belgium,11.37,yes,yes


## 1.2 Subqueries using the SELECT statement

A subquery is a query within another query. The outer query is called as main query and inner query is called as subquery. The subquery must be enclosed with parenthesis and generally executes first, and its output is used to complete the query condition for the main or outer query. We will go through examples of how subqueries can be used to answers different types of questions then later, we will use our learnings to work on the challenges.

In [7]:
# Example 1
# Subqueries can occur in the select list of the containing query
# as shown below;
# Let's find all countries in the Countries table with no city in the Cities table
# 
%%sql
SELECT country
FROM Countries
WHERE NOT EXISTS (SELECT * FROM Cities
                  WHERE Cities.country = Countries.country) 


 * sqlite://
Done.


country
Cyprus
Iceland
Kosovo
Liechtenstein
Luxembourg


In [8]:
# Example 2
# Find the European countries with cities experiencing temperature > 20?
# 
%%sql
SELECT country
FROM Countries
WHERE EU = 'yes'
AND EXISTS (SELECT * FROM Cities
            WHERE Cities.country = Countries.country
            AND temperature > 15)

 * sqlite://
Done.


country
Greece
Italy
Portugal
Spain


In [9]:
# Example 3: Method 1
# Find the westernmost city, returning the city and longitude
# 
%%sql
SELECT city, longitude
FROM Cities C1
WHERE NOT EXISTS (SELECT * FROM Cities C2
                  WHERE C2.longitude < C1.longitude)

 * sqlite://
Done.


city,longitude
Lisbon,-9.14


In [10]:
# Example 3: Method 2
# Find the westernmost city, returning the city and longitude
# 
%%sql
SELECT city, longitude
FROM Cities
WHERE longitude = (SELECT min(longitude) FROM Cities)

 * sqlite://
Done.


city,longitude
Lisbon,-9.14


In [11]:
# Example 4:
# How many cities are in the EU?
# 
%%sql
SELECT count()
FROM Cities
WHERE country IN (SELECT country FROM Countries WHERE EU = 'yes')

 * sqlite://
Done.


count()
150


### <font color="green">1.2 Challenges</font>

In [12]:
# Challenge 1
# Find number of countries that have a city with latitude > 1000 (start with country list)
# 
%%sql
SELECT count()
FROM countries
WHERE country IN (SELECT country FROM cities WHERE latitude > 1000)

 * sqlite://
Done.


count()
0


In [13]:
# Challenge 2
# Find three western most cities returning both the cities and their longitude
#
%%sql
SELECT CITY, LONGITUDE FROM CITIES
ORDER BY LONGITUDE ASC LIMIT 3

 * sqlite://
Done.


city,longitude
Lisbon,-9.14
Galway,-9.05
Vigo,-8.73


In [14]:
# Challenge 3
# Find cities with temperature is more than 50% higher than the average; 
# We will return the city, country, and temperature, ordered by descending temperature
# Hint: avg(temperature) * 1.5
# 
%%sql
SELECT CITY, COUNTRY, TEMPERATURE FROM CITIES
WHERE TEMPERATURE > (SELECT AVG(TEMPERATURE)*1.5 FROM CITIES)
ORDER BY TEMPERATURE DESC

 * sqlite://
Done.


city,country,temperature
Adana,Turkey,18.67
Palermo,Italy,17.9
Athens,Greece,17.41
Algeciras,Spain,17.38
Cartagena,Spain,17.32
Kalamata,Greece,17.3
Marbella,Spain,17.19
Huelva,Spain,17.09
Patras,Greece,16.9
Cosenza,Italy,16.6


In [15]:
# Challenge 4
# How many cities are not in the EU and without a coastline?
# Hint: "not in"
# 
%%sql
SELECT COUNT() FROM CITIES
WHERE COUNTRY NOT IN (SELECT COUNTRY FROM COUNTRIES WHERE EU = "no" AND coastline = "no" )

 * sqlite://
Done.


COUNT()
197


In [16]:
# Challenge 5
# Find the number of countries with no coastline and a city with longitude < 10
# 
%%sql
SELECT COUNT() FROM COUNTRIES
WHERE COASTLINE = "no" AND COUNTRY NOT IN (SELECT COUNTRY FROM CITIES WHERE LONGITUDE < 10)

 * sqlite://
Done.


COUNT()
11


In [18]:
# Challenge 6
# Find countries in Countries table with no city in Cities table
# COMPLETE THE CODE BELOW

%%sql
SELECT country
FROM Countries
WHERE COUNTRY NOT IN (SELECT COUNTRY FROM Cities)

 * sqlite://
Done.


country
Cyprus
Iceland
Kosovo
Liechtenstein
Luxembourg


In [19]:
# Challenge 7
# Find all cities in a country whose population is < 2, returning the city and country.
# 
%%sql
SELECT COUNTRY,CITY FROM CITIES
WHERE COUNTRY IN (SELECT COUNTRY FROM COUNTRIES WHERE POPULATION > 2 )

 * sqlite://
Done.


country,city
Denmark,Aalborg
United Kingdom,Aberdeen
Sweden,Abisko
Turkey,Adana
Spain,Albacete
Spain,Algeciras
France,Amiens
Netherlands,Amsterdam
Italy,Ancona
France,Angers


In [20]:
# Challenge 8
# Find all countries with no city having a temperature > 6
# 
%%sql
SELECT COUNTRY FROM COUNTRIES
WHERE COUNTRY NOT IN (SELECT COUNTRY FROM CITIES WHERE TEMPERATURE >6)

 * sqlite://
Done.


country
Cyprus
Estonia
Finland
Iceland
Kosovo
Latvia
Liechtenstein
Luxembourg
Norway
